In [7]:
# 风险及免责提示：该策略由聚宽用户在聚宽社区分享，仅供学习交流使用。
# 原文一般包含策略说明，如有疑问请到原文和作者交流讨论。
# 原文网址：https://www.joinquant.com/view/community/detail/31231
# 标题：利用聚宽的数据库，研究基金

import numpy as np
import pandas as pd
import datetime as dt
from jqdata import finance

In [8]:
# 指数
index = '000300.XSHG'

In [9]:
# 成分股
stocks = get_index_stocks(index)

In [10]:
# 测试
# 注意：code和symbol的格式，没有后缀！
code = '000001'
df = finance.run_query(query(
        finance.FUND_PORTFOLIO_STOCK
    ).filter(
        finance.FUND_PORTFOLIO_STOCK.symbol==code,
        finance.FUND_PORTFOLIO_STOCK.pub_date > '2020-10-1',
        finance.FUND_PORTFOLIO_STOCK.pub_date < '2021-1-1',
    ))
df[:3]

,id,code,period_start,period_end,pub_date,report_type_id,report_type,rank,symbol,name,shares,market_cap,proportion
0,25043790,000028,2020-07-01,2020-09-30,2020-10-28,403003,第三季度,6,000001,平安银行,37100.0,562807.0,1.15
1,25043801,000029,2020-07-01,2020-09-30,2020-10-28,403003,第三季度,7,000001,平安银行,1167100.0,17704907.0,2.70
2,25044073,000072,2020-07-01,2020-09-30,2020-10-27,403003,第三季度,7,000001,平安银行,239100.0,3627147.0,0.52


In [5]:
# 时间段
dt_start = '2020-10-1'
dt_end = '2021-1-1'

In [6]:
# 逐股提取基金持仓
fportfolio = pd.Series()
for s in stocks:
    code = s[:6]
    df = finance.run_query(query(
            finance.FUND_PORTFOLIO_STOCK
        ).filter(
            finance.FUND_PORTFOLIO_STOCK.symbol==code,
            finance.FUND_PORTFOLIO_STOCK.pub_date > dt_start,
            finance.FUND_PORTFOLIO_STOCK.pub_date < dt_end,
        ))
    total_value = 1e-8*df.market_cap.sum()
    print(code, total_value)
    fportfolio[s] = total_value

000001 138.3619337625
000002 303.9575894742
000063 126.889087858
000066 19.835282625199998
000069 2.9345518727999997
000100 38.634903561
000157 7.7465703817
000166 36.5521303287
000301 0.2624998992
000333 512.6607827519999
000338 33.078054261999995
000408 0.0
000425 5.962880761500001
000538 15.017461072000001
000568 466.24173962849994
000596 109.42123113119999
000625 17.2668884162
000651 161.56487773400002
000661 321.217045224
000703 0.6508448916
000708 12.536560318800001
000725 153.7809378551
000768 48.0838113632
000776 35.08071335459999
000786 29.83262748
000792 0.135992792
000800 0.778678008
000858 1139.13998393
000876 32.36472155
000877 4.4606944663
000895 85.8044879531
000938 9.340688214000002
000963 0.9410764220000001
000977 5.012815305000001
001289 0.0
001979 4.490914803
002001 8.6238987588
002007 91.1825854441
002008 0.887399527
002027 189.65485535579998
002032 22.66488986
002049 141.661785139
002050 114.733435699
002064 0.98530432
002074 4.229448734
002120 7.1536654006
002129 

In [8]:
fportfolio[:3]

000001.XSHE    138.361934
000002.XSHE    303.957589
000063.XSHE    126.889088
dtype: float64

In [9]:
fweight = 100 * fportfolio / fportfolio.sum()
fweight[:3]

000001.XSHE    0.638243
000002.XSHE    1.402111
000063.XSHE    0.585320
dtype: float64

In [10]:
index_weight = get_index_weights(index, dt_start)
index_weight[:3]

,date,weight,display_name
code,,,
000001.XSHE,2020-09-30,0.901,平安银行
000002.XSHE,2020-09-30,1.167,万科A
000063.XSHE,2020-09-30,0.546,中兴通讯


In [15]:
fdelta = pd.Series()
for s in index_weight.index:
    if s in fweight.index:
        fdelta[s] = d = fweight[s] - index_weight.weight[s]

In [16]:
fdelta = fdelta.sort_values(ascending=False)

In [19]:
# 最受欢迎的个股
the_best_delta = fdelta[fdelta > 0.5]
the_best_list = the_best_delta.index.tolist()
the_best = index_weight.loc[the_best_list]
the_best['delta'] = the_best_delta
the_best

,date,weight,display_name,delta
code,,,,
000858.XSHE,2020-09-30,2.625,五粮液,2.629683
601012.XSHG,2020-09-30,1.212,隆基股份,2.194435
002475.XSHE,2020-09-30,1.465,立讯精密,2.186837
600519.XSHG,2020-09-30,5.131,贵州茅台,1.861598
000568.XSHE,2020-09-30,0.643,泸州老窖,1.507704
002352.XSHE,2020-09-30,0.658,顺丰控股,1.232426
601888.XSHG,2020-09-30,1.332,中国中免,1.084300
603259.XSHG,2020-09-30,0.644,药明康德,0.854036
000661.XSHE,2020-09-30,0.641,长春高新,0.840726


In [20]:
# 最被抛弃的个股
the_worst_delta = fdelta[fdelta < -0.5]
the_worst_list = the_worst_delta.index.tolist()
the_worst = index_weight.loc[the_worst_list]
the_worst['delta'] = the_worst_delta
the_worst

,date,weight,display_name,delta
code,,,,
600000.XSHG,2020-09-30,0.675,浦发银行,-0.504159
601166.XSHG,2020-09-30,1.230,兴业银行,-0.508052
603288.XSHG,2020-09-30,0.964,海天味业,-0.516568
600585.XSHG,2020-09-30,0.812,海螺水泥,-0.551368
600016.XSHG,2020-09-30,0.690,民生银行,-0.557556
601328.XSHG,2020-09-30,0.763,交通银行,-0.602636
601398.XSHG,2020-09-30,1.055,工商银行,-0.611310
600900.XSHG,2020-09-30,1.030,长江电力,-0.807628
000651.XSHE,2020-09-30,1.570,格力电器,-0.824725


In [27]:
# 基金持仓变异系数
fportfolio.mean() / fportfolio.std()

0.46624466819291815

In [26]:
# 指数权重变异系数
index_weight.weight.mean() / index_weight.weight.std()

0.6289112833317965

In [31]:
# 同期价格
price = get_price(stocks, dt_start, dt_end, 'daily', 'close', panel=False)
price[:3]

,time,code,close
0,2020-10-09,000001.XSHE,15.18
1,2020-10-12,000001.XSHE,15.90
2,2020-10-13,000001.XSHE,16.06


In [50]:
# 同期收益
Rt = pd.Series()
for s in stocks:
    p = price[price.code == s].close
    Rt[s] = 100*p.iloc[-1] / p.iloc[0] - 100

In [58]:
# 基金组合总收益
1e-2 * np.dot(fweight, Rt)

17.61021401775397

In [57]:
# 指数收益
index_price = get_price(index, dt_start, dt_end, 'daily', 'close', panel=False)
100*index_price.iloc[-1] / index_price.iloc[0] - 100

close    11.325233
dtype: float64